In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import torch
import transformers
import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"
#################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)



logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-05-05 18:26:51 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-05-05 18:26:51 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=4, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-05-05 18:26:51 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
import torch

from src.models import ModelandTokenizer


# model_key = "meta-llama/Llama-3.1-70B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.2-3B"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "google/gemma-3-12b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B"
# model_key = "Qwen/Qwen2.5-32B"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
model_key = "Qwen/Qwen3-14B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

[2025-05-05 18:27:05,999] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-05-05 18:27:06 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpg10ep39p/test.c -o /tmp/tmpg10ep39p/test.o
2025-05-05 18:27:06 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpg10ep39p/test.o -laio -o /tmp/tmpg10ep39p/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-05-05 18:27:06 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpgyndwemw/test.c -o /tmp/tmpgyndwemw/test.o
2025-05-05 18:27:06 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpgyndwemw/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmpgyndwemw/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-05-05 18:27:07 src.models WARNING  Qwen/Qwen3-14B not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-05-05 18:27:07 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-05-05 18:27:07 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen3-14B/resolve/main/config.json HTTP/1.1" 200 0
2025-05-05 18:27:07 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen3-14B/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards: 100%|██████████| 8/8 [00:18<00:00,  2.33s/it]

2025-05-05 18:27:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen3-14B/resolve/main/generation_config.json HTTP/1.1" 200 0


2025-05-05 18:27:27 src.models INFO     loaded model <Qwen/Qwen3-14B> | size: 28168.311 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "trimmed_r1_responses.json"), "r") as f:
    gsm8k_data = json.load(f)

In [9]:
q = gsm8k_data[0]["question"]
prompt = q + "<think>"
print(prompt)

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Please reason step by step, and put your final answer within \boxed{}.<think>


In [12]:
from src.functional import generate_with_patch

generation = generate_with_patch(
    mt = mt,
    inputs = prompt,
    n_gen_per_prompt=1,
    temperature=0.6,
    max_new_tokens=1024,
)[0]

print(generation)

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Please reason step by step, and put your final answer within \boxed{}.<think>
Okay, let's see. Janet has ducks that lay eggs. The problem says they lay 16 eggs per day. Alright, so starting with 16 eggs each day. Then she eats three for breakfast every morning. So, first, I need to subtract the eggs she eats. Let me write that down:

Total eggs laid per day: 16

Eggs eaten for breakfast: 3

So, after eating, the remaining eggs would be 16 - 3. Let me calculate that. 16 minus 3 is 13. So, 13 eggs left after breakfast.

Then, she bakes muffins for her friends every day with four. Wait, does that mean she uses four eggs for baking? The wording is a bit ambiguous. Let me check the original problem again: "bakes mu

In [13]:
messages = [
    {"role": "user", "content": q},
]

text = mt.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

generation = generate_with_patch(
    mt = mt,
    inputs = text,
    n_gen_per_prompt=1,
    temperature=0.6,
    max_new_tokens=1024,
)[0]

print(generation)

user
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Please reason step by step, and put your final answer within \boxed{}.
assistant
<think>
Okay, so Janet has ducks that lay eggs, right? Let me try to figure out how much money she makes each day from selling the eggs at the farmers' market. Let's break it down step by step.

First, the problem says her ducks lay 16 eggs per day. Got that. So starting with 16 eggs each day.

Then, she eats three for breakfast every morning. So that's 3 eggs consumed by her. And then she bakes muffins for her friends every day with four. Wait, does that mean she uses four eggs for baking muffins? The wording says "bakes muffins for her friends every day with four." Hmm, maybe it's four eggs? Or is it four muffins? But since

In [14]:
len(gsm8k_data)

1249